In [17]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [18]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [19]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [20]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [21]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [22]:
df = tripdata[selected_features]

In [23]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 10% NA values

In [24]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc=10):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.loc[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
    return X_train, y_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.loc[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
        y_train.loc[i] = np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [25]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [26]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=10)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=10)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train , y_train = ampute_nan_values(X_train,y_train,variable,perc=10)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values(X_test,y_test,variable,perc=10)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    start = datetime.datetime.now()
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
        
    end = datetime.datetime.now()
    comp_time = (end-start)
    print(f"Time taken simulate for variable {var} = ", comp_time)
    list_comp_time.append(comp_time)         
    return X_train , y_train, X_test , y_test

In [27]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [28]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [29]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [30]:
model_metrics ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  25235
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  6309
X_test Amputed value count of varaible  6309
Simulate Imputation for NA
Train after NA value removal Imputation (227111, 21) (227111,)
Test after NA value removal Imputation (56778, 21) (56778,)
Time taken simulate for variable start_lat =  0:00:00.324513
2023-03-25 12:32:44.729139
2023-03-25 12:32:45.084996
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  25235
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  6309
X_test Amputed value count of varaib

2023-03-25 12:33:56.723907
2023-03-25 12:33:57.048680
Now running model is .... solarradiation
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  25235
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  6309
X_test Amputed value count of varaible  6309
Simulate Imputation for NA
Train after NA value removal Imputation (227111, 21) (227111,)
Test after NA value removal Imputation (56778, 21) (56778,)
Time taken simulate for variable solarradiation =  0:00:00.305118
2023-03-25 12:34:02.488224
2023-03-25 12:34:02.813722
Now running model is .... conditions
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  25235
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed v

In [31]:
print("averge time to run drop is : ", 0.35481595, "seconds")
list_comp_time

averge time to run drop is :  0.350272 seconds


[datetime.timedelta(microseconds=324513),
 datetime.timedelta(microseconds=330419),
 datetime.timedelta(microseconds=340316),
 datetime.timedelta(microseconds=323005),
 datetime.timedelta(microseconds=411985),
 datetime.timedelta(microseconds=372489),
 datetime.timedelta(microseconds=411749),
 datetime.timedelta(microseconds=376130),
 datetime.timedelta(microseconds=334430),
 datetime.timedelta(microseconds=359213),
 datetime.timedelta(microseconds=309658),
 datetime.timedelta(microseconds=390454),
 datetime.timedelta(microseconds=382147),
 datetime.timedelta(microseconds=305118),
 datetime.timedelta(microseconds=338050),
 datetime.timedelta(microseconds=330933),
 datetime.timedelta(microseconds=360098),
 datetime.timedelta(microseconds=388653),
 datetime.timedelta(microseconds=289923),
 datetime.timedelta(microseconds=403114),
 datetime.timedelta(microseconds=368738)]

In [32]:
model_metrics

{'start_lat': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'start_lon': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'end_lat': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'end_lon': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'usertype': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'hour': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'temp': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.5246907136672183},
 'feelslike': {'MAE': 1.5721830060322195,
  'MSE': 5.013337711560819,
  'RMSE': 2.2390483941980395,
  'R2': 0.52

### Mode Replacement

In [33]:
model_metrics_mode ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.264615
2023-03-25 12:34:49.299050
2023-03-25 12:34:49.663098
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lon 

Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable dist =  0:00:00.274763
2023-03-25 12:35:57.647958
2023-03-25 12:35:57.983717
Now model is running for varaible ==== birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable birthyear =  0:00:00.392709
2023-03-25 12:36:02.423711
2023-03-25 12:36:02.819724
Now model is running for varaible ==== holiday
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (2523

In [34]:
print("averge time to run mode is : ", 0.3035832, "seconds")
list_comp_time

averge time to run drop is :  0.350272 seconds


[datetime.timedelta(microseconds=264615),
 datetime.timedelta(microseconds=279423),
 datetime.timedelta(microseconds=295161),
 datetime.timedelta(microseconds=321582),
 datetime.timedelta(microseconds=321874),
 datetime.timedelta(microseconds=286054),
 datetime.timedelta(microseconds=306555),
 datetime.timedelta(microseconds=273831),
 datetime.timedelta(microseconds=280020),
 datetime.timedelta(microseconds=323314),
 datetime.timedelta(microseconds=257496),
 datetime.timedelta(microseconds=290148),
 datetime.timedelta(microseconds=370389),
 datetime.timedelta(microseconds=314179),
 datetime.timedelta(microseconds=276864),
 datetime.timedelta(microseconds=274763),
 datetime.timedelta(microseconds=392709),
 datetime.timedelta(microseconds=314647),
 datetime.timedelta(microseconds=292019),
 datetime.timedelta(microseconds=322829),
 datetime.timedelta(microseconds=316777)]

In [35]:
model_metrics_mode

{'start_lat': {'MAE': 1.5722040548463578,
  'MSE': 4.99969795196476,
  'RMSE': 2.236000436485816,
  'R2': 0.527227699578076},
 'start_lon': {'MAE': 1.5728486211254735,
  'MSE': 5.002883184954267,
  'RMSE': 2.236712584342089,
  'R2': 0.5269265033973674},
 'end_lat': {'MAE': 1.5722304094925525,
  'MSE': 4.999740029191904,
  'RMSE': 2.236009845504242,
  'R2': 0.527223720748222},
 'end_lon': {'MAE': 1.5727225964350673,
  'MSE': 5.000551268379016,
  'RMSE': 2.2361912414592395,
  'R2': 0.527147009830809},
 'usertype': {'MAE': 1.5756771486233472,
  'MSE': 5.021445275137298,
  'RMSE': 2.240858155961081,
  'R2': 0.5251712689490499},
 'hour': {'MAE': 1.5730260790691732,
  'MSE': 5.003754451295777,
  'RMSE': 2.236907340793484,
  'R2': 0.5268441163018841},
 'temp': {'MAE': 1.572541767719938,
  'MSE': 5.00079249478944,
  'RMSE': 2.2362451777006562,
  'R2': 0.5271241994198452},
 'feelslike': {'MAE': 1.5724419588674323,
  'MSE': 5.000249090621296,
  'RMSE': 2.2361236751622875,
  'R2': 0.5271755838116

### Mean Replacement

In [36]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','dew','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

In [37]:
model_metrics_mean ={}
list_comp_time = []
for var in variables_for_mean_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.334002
2023-03-25 12:36:26.611709
2023-03-25 12:36:26.987409
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lon 

In [38]:
print("averge time to run mean is : ", 0.343219, "seconds")
list_comp_time

averge time to run drop is :  0.350272 seconds


[datetime.timedelta(microseconds=334002),
 datetime.timedelta(microseconds=287794),
 datetime.timedelta(microseconds=312079),
 datetime.timedelta(microseconds=269018),
 datetime.timedelta(microseconds=456885),
 datetime.timedelta(microseconds=348609),
 datetime.timedelta(microseconds=315942),
 datetime.timedelta(microseconds=338838),
 datetime.timedelta(microseconds=263303),
 datetime.timedelta(microseconds=543304),
 datetime.timedelta(microseconds=384024),
 datetime.timedelta(microseconds=298874),
 datetime.timedelta(microseconds=298877),
 datetime.timedelta(microseconds=310030),
 datetime.timedelta(microseconds=386714)]

In [39]:
model_metrics_mean

{'start_lat': {'MAE': 1.5722362594683228,
  'MSE': 4.999711471568887,
  'RMSE': 2.2360034596504734,
  'R2': 0.5272264211619786},
 'start_lon': {'MAE': 1.572798121647502,
  'MSE': 5.002659646341988,
  'RMSE': 2.2366626134359175,
  'R2': 0.5269476412470886},
 'end_lat': {'MAE': 1.572254466669696,
  'MSE': 4.999736114555237,
  'RMSE': 2.236008970141944,
  'R2': 0.5272240909169403},
 'end_lon': {'MAE': 1.5727338671514686,
  'MSE': 5.000584346559199,
  'RMSE': 2.2361986375452427,
  'R2': 0.5271438819523873},
 'hour': {'MAE': 1.5726545300386487,
  'MSE': 5.001261216296576,
  'RMSE': 2.236349976255187,
  'R2': 0.52707987703331},
 'temp': {'MAE': 1.5725346409789673,
  'MSE': 5.0007240880455885,
  'RMSE': 2.236229882647486,
  'R2': 0.527130667973339},
 'feelslike': {'MAE': 1.5724659132057233,
  'MSE': 5.00035041131011,
  'RMSE': 2.236146330477974,
  'R2': 0.527166002909878},
 'dew': {'MAE': 1.5722644664869885,
  'MSE': 4.999479622906542,
  'RMSE': 2.2359516146165914,
  'R2': 0.5272483448114669}

### Sampling Replacement

In [40]:
model_metrics_sample ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simple Imputation using sampling
Time taken simulate for variable start_lat =  0:00:00.027097
2023-03-25 12:37:39.993761
2023-03-25 12:37:40.403724
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simple Imputation using sampling
Time taken simulate for variable start_lon =  0:00:00.024000
2023-03-25 12:37:44.306848
2023-03-25 12:37:44.720096
Now model is running for varaible ==== end_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after 

2023-03-25 12:38:57.564547
2023-03-25 12:38:57.881179
Now model is running for varaible ==== month
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simple Imputation using sampling
Time taken simulate for variable month =  0:00:00.115786
2023-03-25 12:39:01.776089
2023-03-25 12:39:02.111965
Now model is running for varaible ==== gender
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simple Imputation using sampling
Time taken simulate for variable gender =  0:00:00.080731
2023-03-25 12:39:05.763256
2023-03-25 12:39:06.146298


In [41]:
print("averge time to run sample is : ",0.4782438, "seconds")
list_comp_time

averge time to run drop is :  0.350272 seconds


[datetime.timedelta(microseconds=27097),
 datetime.timedelta(microseconds=24000),
 datetime.timedelta(microseconds=25000),
 datetime.timedelta(microseconds=23006),
 datetime.timedelta(microseconds=117648),
 datetime.timedelta(microseconds=45998),
 datetime.timedelta(microseconds=40602),
 datetime.timedelta(microseconds=20991),
 datetime.timedelta(microseconds=44389),
 datetime.timedelta(microseconds=23988),
 datetime.timedelta(microseconds=24006),
 datetime.timedelta(microseconds=24684),
 datetime.timedelta(microseconds=23001),
 datetime.timedelta(microseconds=23009),
 datetime.timedelta(microseconds=64063),
 datetime.timedelta(microseconds=25020),
 datetime.timedelta(microseconds=46002),
 datetime.timedelta(microseconds=93506),
 datetime.timedelta(microseconds=91785),
 datetime.timedelta(microseconds=115786),
 datetime.timedelta(microseconds=80731)]

In [42]:
model_metrics_sample

{'start_lat': {'MAE': 1.5721700731898758,
  'MSE': 4.999847797058164,
  'RMSE': 2.2360339436283527,
  'R2': 0.5272135302002063},
 'start_lon': {'MAE': 1.573066088083224,
  'MSE': 5.0040700215225495,
  'RMSE': 2.2369778768513893,
  'R2': 0.5268142759268275},
 'end_lat': {'MAE': 1.572264133773408,
  'MSE': 5.000405173016989,
  'RMSE': 2.2361585751053052,
  'R2': 0.5271608246334331},
 'end_lon': {'MAE': 1.5730148975356515,
  'MSE': 5.0012658589872325,
  'RMSE': 2.23635101426123,
  'R2': 0.527079438019681},
 'usertype': {'MAE': 1.5784133639045246,
  'MSE': 5.035210871002452,
  'RMSE': 2.243927554758052,
  'R2': 0.5238695918304772},
 'hour': {'MAE': 1.5732194712261443,
  'MSE': 5.003413330880514,
  'RMSE': 2.2368310912718723,
  'R2': 0.5268763727071697},
 'temp': {'MAE': 1.5725631169449839,
  'MSE': 5.0009002819574855,
  'RMSE': 2.2362692776044404,
  'R2': 0.5271140070466495},
 'feelslike': {'MAE': 1.5724558949565184,
  'MSE': 5.000288520207748,
  'RMSE': 2.236132491648862,
  'R2': 0.527171